In [6]:
import random

# Constants for the board and players
EMPTY, BLACK, WHITE = 0, 1, 2
DIRECTIONS = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]
BOARD_SIZE = 8

class OthelloGame:
    def __init__(self):
        self.board = [[EMPTY] * BOARD_SIZE for _ in range(BOARD_SIZE)]
        self.board[3][3], self.board[4][4] = WHITE, WHITE
        self.board[3][4], self.board[4][3] = BLACK, BLACK
        self.current_player = BLACK

    def print_board(self):
        print("  " + " ".join(str(i) for i in range(BOARD_SIZE)))
        for i in range(BOARD_SIZE):
            print(i, end=" ")
            for j in range(BOARD_SIZE):
                if self.board[i][j] == BLACK:
                    print("B", end=" ")
                elif self.board[i][j] == WHITE:
                    print("W", end=" ")
                else:
                    print(".", end=" ")
            print()

    def valid_move(self, row, col, player):
        if self.board[row][col] != EMPTY:
            return False
        opponent = WHITE if player == BLACK else BLACK
        for dr, dc in DIRECTIONS:
            r, c = row + dr, col + dc
            flipped = False
            while 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE and self.board[r][c] == opponent:
                r += dr
                c += dc
                flipped = True
            if flipped and 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE and self.board[r][c] == player:
                return True
        return False

    def make_move(self, row, col, player):
        opponent = WHITE if player == BLACK else BLACK
        self.board[row][col] = player
        for dr, dc in DIRECTIONS:
            r, c = row + dr, col + dc
            flipped_positions = []
            while 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE and self.board[r][c] == opponent:
                flipped_positions.append((r, c))
                r += dr
                c += dc
            if flipped_positions and 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE and self.board[r][c] == player:
                for fr, fc in flipped_positions:
                    self.board[fr][fc] = player

    def get_valid_moves(self, player):
        return [(r, c) for r in range(BOARD_SIZE) for c in range(BOARD_SIZE) if self.valid_move(r, c, player)]

    def switch_player(self):
        self.current_player = WHITE if self.current_player == BLACK else BLACK

    def count_pieces(self):
        black_count = sum(row.count(BLACK) for row in self.board)
        white_count = sum(row.count(WHITE) for row in self.board)
        return black_count, white_count

    def is_game_over(self):
        return not self.get_valid_moves(BLACK) and not self.get_valid_moves(WHITE)

    # Strategy 1: Random move
    def random_strategy(self):
        valid_moves = self.get_valid_moves(self.current_player)
        if valid_moves:
            return random.choice(valid_moves)
        return None

    # Strategy 2: Maximize immediate gains
    def greedy_strategy(self):
        valid_moves = self.get_valid_moves(self.current_player)
        if valid_moves:
            max_flips = -1
            best_move = None
            for move in valid_moves:
                row, col = move
                count_flips = self.simulate_flips(row, col, self.current_player)
                if count_flips > max_flips:
                    max_flips = count_flips
                    best_move = move
            return best_move
        return None

    # Strategy 3: Minimize opponent's next move
    def minimize_opponent_strategy(self):
        valid_moves = self.get_valid_moves(self.current_player)
        if valid_moves:
            min_opponent_moves = float('inf')
            best_move = None
            for move in valid_moves:
                row, col = move
                self.make_move(row, col, self.current_player)
                opponent_moves = len(self.get_valid_moves(WHITE if self.current_player == BLACK else BLACK))
                if opponent_moves < min_opponent_moves:
                    min_opponent_moves = opponent_moves
                    best_move = move
                self.undo_move(row, col, self.current_player)
            return best_move
        return None

    # Helper function to simulate number of flips for a given move
    def simulate_flips(self, row, col, player):
        opponent = WHITE if player == BLACK else BLACK
        count = 0
        for dr, dc in DIRECTIONS:
            r, c = row + dr, col + dc
            flipped_positions = []
            while 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE and self.board[r][c] == opponent:
                flipped_positions.append((r, c))
                r += dr
                c += dc
            if flipped_positions and 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE and self.board[r][c] == player:
                count += len(flipped_positions)
        return count

    # Undo the move (for simulation purposes)
    def undo_move(self, row, col, player):
        opponent = WHITE if player == BLACK else BLACK
        self.board[row][col] = EMPTY
        for dr, dc in DIRECTIONS:
            r, c = row + dr, col + dc
            flipped_positions = []
            while 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE and self.board[r][c] == player:
                flipped_positions.append((r, c))
                r += dr
                c += dc
            if flipped_positions and 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE and self.board[r][c] == opponent:
                for fr, fc in flipped_positions:
                    self.board[fr][fc] = opponent

# Main game loop
def play_game(strategy_func):
    game = OthelloGame()
    human_player = BLACK  # You are black
    ai_player = WHITE
    while not game.is_game_over():
        game.print_board()
        if game.current_player == human_player:
            valid_moves = game.get_valid_moves(human_player)
            if valid_moves:
                print("Your valid moves:", valid_moves)
                row, col = map(int, input("Enter your move (row, col): ").split())
                if (row, col) in valid_moves:
                    game.make_move(row, col, human_player)
                else:
                    print("Invalid move! Try again.")
                    continue
        else:
            move = strategy_func(game)
            if move:
                print(f"AI move: {move}")
                game.make_move(move[0], move[1], ai_player)
            else:
                print("AI has no valid moves!")
        game.switch_player()

    game.print_board()
    black_count, white_count = game.count_pieces()
    print(f"Final Score - Black: {black_count}, White: {white_count}")
    if black_count > white_count:
        print("You win!")
    elif white_count > black_count:
        print("AI wins!")
    else:
        print("It's a tie!")

# Choose strategy
print("Choose AI strategy:")
print("1. Random Strategy")
print("2. Greedy Strategy (Maximize Immediate Gains)")
print("3. Minimize Opponent Strategy")
strategy_choice = input("Enter your choice (1/2/3): ")

if strategy_choice == '1':
    strategy = OthelloGame.random_strategy
elif strategy_choice == '2':
    strategy = OthelloGame.greedy_strategy
elif strategy_choice == '3':
    strategy = OthelloGame.minimize_opponent_strategy
else:
    print("Invalid choice! Defaulting to random strategy.")
    strategy = OthelloGame.random_strategy

# Play the game
play_game(strategy)

Choose AI strategy:
1. Random Strategy
2. Greedy Strategy (Maximize Immediate Gains)
3. Minimize Opponent Strategy
  0 1 2 3 4 5 6 7
0 . . . . . . . . 
1 . . . . . . . . 
2 . . . . . . . . 
3 . . . W B . . . 
4 . . . B W . . . 
5 . . . . . . . . 
6 . . . . . . . . 
7 . . . . . . . . 
Your valid moves: [(2, 3), (3, 2), (4, 5), (5, 4)]
  0 1 2 3 4 5 6 7
0 . . . . . . . . 
1 . . . . . . . . 
2 . . . B . . . . 
3 . . . B B . . . 
4 . . . B W . . . 
5 . . . . . . . . 
6 . . . . . . . . 
7 . . . . . . . . 
AI move: (2, 2)
  0 1 2 3 4 5 6 7
0 . . . . . . . . 
1 . . . . . . . . 
2 . . W B . . . . 
3 . . . W B . . . 
4 . . . B W . . . 
5 . . . . . . . . 
6 . . . . . . . . 
7 . . . . . . . . 
Your valid moves: [(2, 1), (3, 2), (4, 5), (5, 4)]


ValueError: not enough values to unpack (expected 2, got 0)